In [ ]:
%pip install anomalib
%pip install lightning kornia torchmetrics torch FrEIA python-dotenv open-clip-torch

# INTRO:
Anomaly detection helps identify defects in products which can significantly improve quality control. This work aims to evaluate two anomaly detection models: PatchCore and EfficientAD. The dataset we will be using is the MVTec-AD dataset using the anomalib library. Our goal is to assess each model’s ability in detecting anomalies across various product categories, reporting Area Under the Receiver Operating Characteristic(auroc) scores both at the category level and as an overall average.


# Information and Methdology
Information regarding The MVTec-AD dataset: The dataset consist of several industrial product categories. Each element has a range of normal and anomalous images. The categories tile, leather, and grid have been selected for this study to analyze their performance on relatively flat surfaces. AUROC was chosen as the primary evaluation metric because it provides a comprehensive measure of model accuracy in binary classification tasks by plotting the true positive rate against the false positive rate.

Information regarding the Anomalib Library: Anomalib is an open-source library designed to simplify anomaly detection research and deployment. We will be using it's pre-trained models and streamlined evaluation tools, to test and assess anomaly detection methods.



# EfficientAD
The EfficientAD model is a one-class anomaly detector that uses features extracted from a convolutional neural network (CNN) to distinguish between normal and anomalous images. We train the model using only normal images. To detect local anomalies, the detector operates on the student teacher model. This is where the teacher is a pretrained deep neural network that understands normal data behavior, and the student is a smaller network that learns to predict the features of normal images output by the teacher model. When given a image both the student and teacher evaluate the image data, and the model detects anomalies using the discrepancy between the student and teacher when the student fails to predict the anomaly features. In addition to structural anomalies, EfficientAD can detect logical anomalies that result from incorrect object orientation by using an autoencoder that is tuned to detect global anomalies. The high accuracy, low latency, and high throughput of EfficientAD allow for you to be able to use this model in real-time scenarios.

In [94]:
from anomalib.data import MVTec
from anomalib.engine import Engine
from anomalib.models import EfficientAd
import shutil
import os

categories = ['tile', 'leather', 'grid']
for category in categories:
    dataset_path = os.path.join(os.path.expanduser("~"), ".anomalib", "datasets", "MVTec", category)
    if os.path.exists(dataset_path):
        shutil.rmtree(dataset_path)
        print(f"Deleted existing dataset folder for category: {category}")

datamodule_tile = MVTec(category='tile', train_batch_size=1)
datamodule_leather = MVTec(category='leather', train_batch_size=1)
datamodule_grid = MVTec(category='grid', train_batch_size=1)

# runs too slow for the google free tier
#EfficientAd takes 40minutes
efficientad_model = EfficientAd()

efficientad_engine = Engine(max_epochs=1)

# Fitting the EfficientAd model to the tile dataset

In [95]:
efficientad_engine.fit(datamodule=datamodule_tile, model=efficientad_model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead

  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model                 | EfficientAdModel         | 8.1 M  | train
1 | _transform            | Compose                  | 0      | train
2 | normalization_metrics | MetricCollection         | 0      | train
3 | image_threshold       | F1AdaptiveThreshold      | 0      | train
4 | pixel_threshold       | F1AdaptiveThreshold      | 0      | train
5 | image_metrics         | AnomalibMetricCollection | 0      | train
6 | pixel_metrics         | AnomalibMetricCollection | 0    

Training: |          | 0/? [00:00<?, ?it/s]

/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureW

Epoch 0:   0%|          | 0/230 [00:00<?, ?it/s] 


/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/core/module.py:516: You called `self.log('train_st', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/core/module.py:516: You called `self.log('train_ae', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/core/module.py:516: You called `self.log('train_stae', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/core/module.py:516: You called `self.log('train_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(..

Epoch 0: 100%|██████████| 230/230 [00:39<00:00,  5.86it/s, train_st_step=11.00, train_ae_step=0.920, train_stae_step=0.000709, train_loss_step=11.90]

Calculate Validation Dataset Quantiles:   0%|          | 0/4 [00:00<?, ?it/s]/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370cours

Epoch 0: 100%|██████████| 230/230 [01:52<00:00,  2.05it/s, train_st_step=11.00, train_ae_step=0.920, train_stae_step=0.000709, train_loss_step=11.90, pixel_AUROC=0.764, pixel_F1Score=0.436, train_st_epoch=12.80, train_ae_epoch=1.010, train_stae_epoch=0.000934, train_loss_epoch=13.80]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 230/230 [01:52<00:00,  2.04it/s, train_st_step=11.00, train_ae_step=0.920, train_stae_step=0.000709, train_loss_step=11.90, pixel_AUROC=0.764, pixel_F1Score=0.436, train_st_epoch=12.80, train_ae_epoch=1.010, train_stae_epoch=0.000934, train_loss_epoch=13.80]


# Fitting the EfficientAd model to the leather dataset

In [96]:
efficientad_engine.fit(datamodule=datamodule_leather, model=efficientad_model)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/junior/Desktop/cs370course/cs370/assignments/assignment-2b/results/EfficientAd/MVTec/tile/v2/weights/lightning exists and is not empty.

  | Name                     | Type                     | Params | Mode 
------------------------------------------------------------------------------
0 | model                    | EfficientAdModel         | 8.1 M  | train
1 | _transform               | Compose                  | 0      | train
2 | normalization_metrics    | MetricCollection         | 0      | train
3 | image_threshold          | F1AdaptiveThreshold      | 0      | train
4 | pixel_

# Fitting the EfficientAd model to the grid dataset

In [97]:
efficientad_engine.fit(datamodule=datamodule_grid, model=efficientad_model)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/junior/Desktop/cs370course/cs370/assignments/assignment-2b/results/EfficientAd/MVTec/tile/v2/weights/lightning exists and is not empty.

  | Name                     | Type                     | Params | Mode 
------------------------------------------------------------------------------
0 | model                    | EfficientAdModel         | 8.1 M  | train
1 | _transform               | Compose                  | 0      | train
2 | normalization_metrics    | MetricCollection         | 0      | train
3 | image_threshold          | F1AdaptiveThreshold      | 0      | train
4 | pixel_

# Testing the EfficientAd model to the 3 selected dataset

In [98]:
from anomalib.metrics import AUROC
test_results_tile = efficientad_engine.test(datamodule=datamodule_tile, model=efficientad_model)
test_results_leather = efficientad_engine.test(datamodule=datamodule_leather, model=efficientad_model)
test_results_grid = efficientad_engine.test(datamodule=datamodule_grid, model=efficientad_model)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureW

Testing DataLoader 0: 100%|██████████| 4/4 [00:55<00:00,  0.07it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9520202279090881     │
│       image_F1Score       │     0.916201114654541     │
│        pixel_AUROC        │     0.764315128326416     │
│       pixel_F1Score       │    0.5193252563476562     │
└───────────────────────────┴───────────────────────────┘

...

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead


Testing: |          | 0/? [00:00<?, ?it/s]

/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureW

Testing DataLoader 0: 100%|██████████| 3/3 [00:30<00:00,  0.10it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.7234753966331482     │
│       image_F1Score       │    0.8444444537162781     │
│        pixel_AUROC        │    0.42092692852020264    │
│       pixel_F1Score       │   0.016385024413466454    │
└───────────────────────────┴───────────────────────────┘

# AUROC Score

In [100]:
from anomalib.metrics import AUROC
auroc_score_tile = test_results_tile[0]['image_AUROC']
auroc_score_leather = test_results_leather[0]['image_AUROC']
auroc_score_grid = test_results_grid[0]['image_AUROC']

print(f"AUROC Score for tile: {auroc_score_tile}")
print(f"AUROC Score for leather: {auroc_score_leather}")
print(f"AUROC Score for grid: {auroc_score_grid}")
print(f"Average AUROC Score: {(auroc_score_grid+auroc_score_tile+auroc_score_leather)/3}")


AUROC Score for tile: 0.9520202279090881
AUROC Score for leather: 0.5
AUROC Score for grid: 0.7234753966331482
Average AUROC Score: 0.7251652081807455


# Patchcore
The PatchCore algorithm leverages coresets to improve anomaly detection performance by reducing the data representation to a subset of important samples. Coreset is generally a weighted and smaller subset of the initial dataset, which provides the same solution as solving the problem on the complete and larger dataset without affecting performance metrics. Coresets help make the model more efficient by retaining only the most important data points, which boost both  accuracy and efficiency. For our problem below, the coreset involved a long compute so he kept it to a minimum. You can increase the coreset_sampling_ratio higher if you have more compute.



In [68]:
from anomalib.data import MVTec
from anomalib.engine import Engine
from anomalib.models import Patchcore
import matplotlib.pyplot as plt
import shutil
import os

categories = ['tile', 'leather', 'grid']
for category in categories:
    dataset_path = os.path.join(os.path.expanduser("~"), ".anomalib", "datasets", "MVTec", category)
    if os.path.exists(dataset_path):
        shutil.rmtree(dataset_path)
        print(f"Deleted existing dataset folder for category: {category}")

#uncomment to run for the other categories, runs too slow for the google free tier
datamodule_tile = MVTec(category='tile', train_batch_size=1)
datamodule_leather = MVTec(category='leather', train_batch_size=1)
datamodule_grid = MVTec(category='grid', train_batch_size=1)

#making the model, patchcore will take 2-3hrs even with disabling coreset sampling(coreset_sampling_ratio=0.0)
#can edit the number or get rid of the parameter if you have enough compute
patchcore_model = Patchcore(coreset_sampling_ratio=0.1)
patchcore_engine = Engine(max_epochs=1)


# Fitting patchcore to tile dataset

In [69]:
patchcore_engine.fit(datamodule=datamodule_tile, model=patchcore_model)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 24.9 M | train
1 | _transform            | Compose                  | 0      | train
2 | normalization_metrics | MetricCollection         | 0      | train
3 | image_threshold       | F1AdaptiveThreshold      | 0      | train
4 | pixel_threshold       | F1AdaptiveThreshold      | 0      | train
5 | image_metrics         | AnomalibMetric

Epoch 0:   0%|          | 0/230 [00:00<?, ?it/s] 

/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureW

Epoch 0: 100%|██████████| 230/230 [00:14<00:00, 16.02it/s]


/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: Future

Epoch 0: 100%|██████████| 230/230 [02:40<00:00,  1.44it/s, pixel_AUROC=0.502, pixel_F1Score=0.153]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 230/230 [02:40<00:00,  1.43it/s, pixel_AUROC=0.502, pixel_F1Score=0.153]


# Fitting patchcore to leather dataset

In [70]:
patchcore_engine.fit(datamodule=datamodule_leather, model=patchcore_model)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/junior/Desktop/cs370course/cs370/assignments/assignment-2b/results/Patchcore/MVTec/tile/v1/weights/lightning exists and is not empty.
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 24.9 M | train
1 | _transform            | Compose  

# Fitting patchcore to grid dataset

In [71]:
patchcore_engine.fit(datamodule=datamodule_grid, model=patchcore_model)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/junior/Desktop/cs370course/cs370/assignments/assignment-2b/results/Patchcore/MVTec/tile/v1/weights/lightning exists and is not empty.
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/core/optimizer.py:182: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params | Mode 
---------------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 24.9 M | train
1 | _transform            | Compose  

# Testing the Patchcore model to the 3 selected dataset

In [92]:
from anomalib.metrics import AUROC
test_results_tile = patchcore_engine.test(datamodule=datamodule_tile, model=patchcore_model)
test_results_leather = patchcore_engine.test(datamodule=datamodule_leather, model=patchcore_model)
test_results_grid = patchcore_engine.test(datamodule=datamodule_grid, model=patchcore_model)


F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:419: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureW

Testing DataLoader 0: 100%|██████████| 4/4 [00:53<00:00,  0.08it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │    0.9877344369888306     │
│       image_F1Score       │    0.9818181991577148     │
│        pixel_AUROC        │    0.9473768472671509     │
│       pixel_F1Score       │    0.6202718019485474     │
└───────────────────────────┴───────────────────────────┘

...

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead


Testing: |          | 0/? [00:00<?, ?it/s]

/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
/Users/junior/Desktop/cs370course/my_env/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureW

Testing DataLoader 0: 100%|██████████| 3/3 [00:32<00:00,  0.09it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │            0.5            │
│       image_F1Score       │    0.8444444537162781     │
│        pixel_AUROC        │    0.26238134503364563    │
│       pixel_F1Score       │   0.016496727243065834    │
└───────────────────────────┴───────────────────────────┘

# AUROC Score

In [73]:
from anomalib.metrics import AUROC
auroc_score_tile = test_results_tile[0]['image_AUROC']
auroc_score_leather = test_results_leather[0]['image_AUROC']
auroc_score_grid = test_results_grid[0]['image_AUROC']

print(f"AUROC Score for tile: {auroc_score_tile}")
print(f"AUROC Score for leather: {auroc_score_leather}")
print(f"AUROC Score for grid: {auroc_score_grid}")
print(f"Average AUROC Score: {(auroc_score_grid+auroc_score_tile+auroc_score_leather)/3}")


AUROC Score for tile: 0.9877344369888306
AUROC Score for leather: 0.712296187877655
AUROC Score for grid: 0.5
Average AUROC Score: 0.7333435416221619


# Result
The results show that PatchCore generally achieved higher AUROC scores across categories, indicating that its use of coresets effectively enhances anomaly detection capabilities on the 3 dataset of anomalies. While less accurate, EfficientAD demonstrated solid performance and may be more suitable for real-time processing as it prioritized speed over peak accuracy. If accuracy is of utmost important and mislabeling an image can cause serious problems, such as the medical case of a mislabeling a healthy patient with a tumor, then using PatchCore is preferred. One thing to note is that pacthcore's AUROC scores could possibly be higher, since I was limited by compute, fitting the model took longer and was less reliable compared to using a higher coreset_sampling_ratio. Despite, this fact the patchcore still performed better than the other model.

Below I tried to visualize the results for the AUROC, but kept running into some errors regarding my device. 

In [75]:
# import torch

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# datamodule_tile.setup(stage="test")
# predictions, labels = [], []

# for batch in datamodule_tile.test_dataloader():
#   if "image" in batch:
#     image = batch["image"]
#     output = efficientad_model(image)  
#     if output is not None:
#         # print(output)
#         predictions.extend(output["predictions"].cpu().numpy())
#         labels.extend(batch["label"].cpu().numpy())


# # Calculate AUROC
# auroc_score = roc_auc_score(labels, predictions)
# print(f"AUROC Score for EfficientAD on 'tile' category: {auroc_score}")

# # Plot the ROC Curve
# fpr, tpr, _ = roc_curve(labels, predictions)

# plt.figure(figsize=(8, 6))
# plt.plot(fpr, tpr, lw=2, label=f'AUC = {auroc_score}')
# plt.plot([0, 1], [0, 1], lw=2, linestyle='--')
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.title("ROC Curve for EfficientAD on tile Category")
# plt.legend()
# plt.show()